## 1.首先对2个CSV需要去重，否则无法合并

In [4]:
# 处理重复索引标签【备用列表-trade_date日期 - ts_code】
import pandas as pd

# 1. 加载CSV文件到DataFrame
df = pd.read_csv('备用列表.csv')

# 2. 筛选并保留20240717日期的数据
df_cleaned = df[df['trade_date'] == 20240717]

# 3. 删除重复的ts_code条目，保留最新数据
df_cleaned = df_cleaned.sort_values('trade_date').drop_duplicates('ts_code', keep='last')

df_cleaned.to_csv('去重备用列表.csv', index=False)


In [5]:
# 曾用名去重 并 保留最新的（更名公告日期）记录【股票曾用名-ann_date公告日 - ts_code】
# 此代码通过按照每个ts code的ann date进行排序和去重，确保每个公司的最新重要公告日期信息得到保留且是唯一的。
import pandas as pd

df = pd.read_csv('股票曾用名.csv')

# 按 ts_code 分组，按 ann_date 降序排序，保留最新的记录
df_sorted = df.sort_values(by=['ts_code', 'ann_date'], ascending=[True, False])
df_deduplicated = df_sorted.drop_duplicates(subset=['ts_code'], keep='first')
# 将处理后的数据保存到新的 CSV 文件中
df_deduplicated.to_csv('去重曾用名.csv', index=False)



## 合并csv（未清洗）

In [7]:
import pandas as pd

# 定义文件列表
files = [
    '去重备用列表.csv',
    '股票列表.csv',
    '去重曾用名.csv',
    'IPO新股上市.csv',
    '上市公司基本信息.csv'
]

# 读取所有文件并设置ts_code作为索引，存储在一个字典中
dfs = {file: pd.read_csv(file, index_col='ts_code') for file in files}

# 合并所有DataFrame，按索引进行外连接
merged_df = pd.concat(dfs.values(), axis=1)

# 保存合并后的数据
merged_df.to_csv('基础数据_未清洗.csv')
print(merged_df.head())

           trade_date   name industry area     pe  float_share  total_share  \
ts_code                                                                       
301552.SZ  20240717.0   科力装备     汽车配件   河北   0.00         0.00         0.00   
002243.SZ  20240717.0   力合科创     广告包装   深圳   0.00        11.96        12.11   
002219.SZ  20240717.0    新里程     医疗保健   甘肃  61.84        32.90        34.11   
002191.SZ  20240717.0   劲嘉股份     广告包装   深圳  11.09        14.26        14.62   
002181.SZ  20240717.0  粤 传 媒      出版业   广东   0.00        11.34        11.61   

           total_assets  liquid_assets  fixed_assets  ...  reg_capital  \
ts_code                                               ...                
301552.SZ          6.53           4.80          0.86  ...          NaN   
002243.SZ        156.64          70.26         16.39  ...  121060.4219   
002219.SZ         61.88          20.27         18.89  ...  341075.8753   
002191.SZ         92.06          38.02         16.33  ...  146207.6254   
00

### 浏览合并结果

In [36]:
import pandas as pd
# 查看详细信息
# 读取CSV文件
df = pd.read_csv('基础数据_未清洗.csv')  

# 查看前几行数据
print(df.head())

# 查看数据的基本信息（包括行数、列数、每列的数据类型和非空值数量）
print(df.info())

# 查看所有列名
print(df.columns)

# 描述性统计信息
print(df.describe(include='all'))


     ts_code  trade_date   name industry area     pe  float_share  \
0  301552.SZ  20240717.0   科力装备     汽车配件   河北   0.00         0.00   
1  002243.SZ  20240717.0   力合科创     广告包装   深圳   0.00        11.96   
2  002219.SZ  20240717.0    新里程     医疗保健   甘肃  61.84        32.90   
3  002191.SZ  20240717.0   劲嘉股份     广告包装   深圳  11.09        14.26   
4  002181.SZ  20240717.0  粤 传 媒      出版业   广东   0.00        11.34   

   total_share  total_assets  liquid_assets  ...  reg_capital  setup_date  \
0         0.00          6.53           4.80  ...          NaN         NaN   
1        12.11        156.64          70.26  ...  121060.4219  19950714.0   
2        34.11         61.88          20.27  ...  341075.8753  20010930.0   
3        14.62         92.06          38.02  ...  146207.6254  19961014.0   
4        11.61         52.77          10.83  ...  116105.8174  19921228.0   

   province  city                                       introduction  \
0       NaN   NaN                                 

## 2.数据清理
#### 清理行/退市/去重、提取列

In [2]:
import pandas as pd

# 读取 CSV 文件并跳过第一行
df = pd.read_csv('基础数据_未清洗.csv')

# 仅保留以 '6', '3', '0' 开头的代码
filtered_df = df[df['ts_code'].str.startswith(('6', '3', '0'))]

# 清理标签 'area'地区 或 'industry' 为空值的行
df_cleaned = filtered_df.dropna(subset=['area'])

# 去除重复的行，保留最后一个出现的
df_cleaned = df_cleaned.drop_duplicates(subset='ts_code', keep='last')

# 定义所需的列，并按照给定顺序排列
desired_columns = [
    'ts_code', 'trade_date', 'name', 'industry', 'fullname', 'enname', 'area', 'city', 
    'list_date', 'start_date', 'change_reason', 'is_hs', 'act_name', 'act_ent_type', 
    'chairman', 'manager', 'setup_date', 'reg_capital', 'business_scope', 'employees', 'introduction', 
    'website', 'email', 'main_business', 'pe', 'float_share', 'total_share', 
    'total_assets', 'liquid_assets', 'fixed_assets', 'reserved', 'eps', 'bvps', 'pb', 
    'undp', 'per_undp', 'rev_yoy', 'profit_yoy', 'gpr', 'npr', 'holder_num'
]

# 提取所需列
df_final = df_cleaned[desired_columns]

# 保存结果到新的 CSV 文件
df_final.to_csv('1基础数据_预处理.csv', index=False)


#### （备用操作）去重 并 添加指定“标签列” （备用操作）

In [46]:
import pandas as pd

# 读取第一个 CSV 文件，并确保 'ts_code' 列没有重复
df1 = pd.read_csv('股票曾用名.csv')
df1.drop_duplicates(subset='ts_code', keep='last', inplace=True)

# 读取第二个 CSV 文件，并确保 'ts_code' 列没有重复（跳过首行中文）
df2 = pd.read_csv('x.csv')
df2.drop_duplicates(subset='ts_code', keep='last', inplace=True)

# 合并两个 DataFrame，基于 'ts_code' 列进行合并
merged_df = pd.merge(df2, df1[['ts_code', 'change_reason']], on='ts_code', how='left')

# 解决重复的 `ts_code`
# 如果你要保留某些特定列的唯一值，例如 'trade_date' 最新的记录，可以使用以下方法：
merged_df = merged_df.sort_values(by='trade_date').drop_duplicates(subset='ts_code', keep='last')

# 将 'change_reason' 列插入到 'ann_date' 列的后面
ann_date_index = merged_df.columns.get_loc('ann_date') + 1  # 获取 'ann_date' 列的索引并加1
change_reason = merged_df.pop('change_reason')  # 弹出 'change_reason' 列
merged_df.insert(ann_date_index, 'change_reason', change_reason)  # 插入 'change_reason' 列

# 保存合并后的 DataFrame 到新的 CSV 文件
merged_df.to_csv('去重加标签列.csv', index=False)


#### 常用方法：去重、去st

In [41]:
import pandas as pd

df = pd.read_csv('基础数据_预处理.csv', skiprows=[0])

# 去除 ts_code 标签重复的行，保留最后一个出现的
# df.drop_duplicates(subset='ts_code', keep='last', inplace=True)

# 去除标签列 name 中含有 ST 或 *ST 的所有行
df_filtered = df[~df['name'].str.contains('ST|\*ST')]

# 将处理后的数据保存到 CSV 文件
df_filtered.to_csv('基础数据_去st.csv', index=False)

### 3.关键词检索

In [4]:
import pandas as pd

# 读取CSV文件
df = pd.read_csv('基础数据_预处理.csv')

# 定义关键词：变量
keywords = ["出租车", "出租", "车"]

def match_keywords(text, keywords):
    # 统计匹配到的关键词数
    return sum(1 for keyword in keywords if keyword in text)

# 在公司简介和主营业务列中分别匹配关键词并统计匹配数
df['公司简介匹配数'] = df['introduction'].fillna('').apply(lambda x: match_keywords(x, keywords))
df['主营业务匹配数'] = df['main_business'].fillna('').apply(lambda x: match_keywords(x, keywords))

# 筛选出公司简介或主营业务中匹配到至少两个关键词的行
filtered_df = df[(df['公司简介匹配数'] >= 2) | (df['主营业务匹配数'] >= 2)]

# 输出结果的形状
print("搜索到的公司数量:", filtered_df.shape[0])

# 输出所有筛选结果的前五列
print(filtered_df.iloc[:, :5])

filtered_df.to_csv('搜索到的公司.csv', index=False)

搜索到的公司数量: 22
        ts_code  trade_date   name industry            fullname
63    000978.SZ  20240717.0   桂林旅游     旅游景点          桂林旅游股份有限公司
494   600635.SH  20240717.0   大众公用     供气供热  上海大众公用事业(集团)股份有限公司
1010  600611.SH  20240717.0   大众交通     公共交通      大众交通(集团)股份有限公司
1045  600530.SH  20240717.0   交大昂立     医疗保健        上海交大昂立股份有限公司
1216  000906.SZ  20240717.0   浙商中拓     仓储物流        浙商中拓集团股份有限公司
1441  600692.SH  20240717.0   亚通股份     区域地产          上海亚通股份有限公司
1589  603069.SH  20240717.0   海汽集团     公共交通      海南海汽运输集团股份有限公司
1670  600676.SH  20240717.0   交运股份     汽车配件        上海交运集团股份有限公司
1674  600620.SH  20240717.0   天宸股份      综合类         上海市天宸股份有限公司
1678  600662.SH  20240717.0   外服控股     文教休闲      上海外服控股集团股份有限公司
1682  000421.SZ  20240717.0   南京公用     供气供热        南京公用发展股份有限公司
1684  600650.SH  20240717.0   锦江在线     公共交通    上海锦江在线网络服务股份有限公司
1768  600007.SH  20240717.0   中国国贸     园区开发      中国国际贸易中心股份有限公司
2245  600009.SH  20240717.0   上海机场       机场        上海国际机场股份有限公司
2410  000011.SZ  20240717.0